# 数据预处理

## 导入文件

In [83]:
import pandas as pd

df = pd.read_csv('train_10000.csv')

检查数据的形状是否正确

In [84]:
# df.head()

调用函数 info() 可以获得有关这个数据集的信息

In [85]:
# df.info()

调用函数 barh() 绘制出 label 数据的条形图

In [86]:
# df.label.value_counts().plot.barh()

检查有效数据

In [87]:
# label_0_df = df[(df.label == 0)]
# label_1_df = df[(df.label == 1)]
# label_2_df = df[(df.label == 2)]
# label_3_df = df[(df.label == 3)]
# label_4_df = df[(df.label == 4)]
#
# print(f'label 0 df: {label_0_df.shape}')
# print(f'label 1 df: {label_1_df.shape}')
# print(f'label 2 df: {label_2_df.shape}')
# print(f'label 3 df: {label_3_df.shape}')
# print(f'label 4 df: {label_4_df.shape}')

绘制图形观察规律

散点图看不出来

In [88]:
# fig, axs = plt.subplots(nrows=12, ncols=9, figsize=(100, 100))
#
# for i in range(12):
#     for j in range(9):
#         axs[i, j].scatter(df.iloc[:, i * 9 + j], df.iloc[:, -1])
#         axs[i, j].set_title(f"Column {i * 9 + j + 1}")
#
# plt.show()

重叠折线图

In [89]:
# from matplotlib import pyplot as plt
#
# df_sample = df.groupby('label').apply(lambda x: x.sample(n=1))
# plt.figure(figsize=(10, 10))
# plt.grid()
# for index, row in df_sample.iterrows():
#     features = row[1:]
#     plt.plot(features[0])
# plt.xticks(range(107), range(107))
# plt.show()

每个feature一个图，x是行数，y是所在值

In [90]:
# from matplotlib import pyplot as plt
#
# rows = 11
# cols = 10
# fig, axs = plt.subplots(nrows=rows, ncols=cols, figsize=(200, 220))
#
# feature = df.drop(['sample_id'], axis=1)
# feature.sort_values(by=['label'], inplace=True)
# feature.reset_index(drop=True, inplace=True)
#
# colors = ['red', 'blue', 'green', 'purple', 'orange', 'magenta']
# for i in range(feature.shape[1] - 1):
#     axs_item = axs[i // cols][i % cols]
#     for j in range(6):
#         axs_item.plot(feature.iloc[:, i][feature.iloc[:, -1] == j], color=colors[j], label=f'Label {j}')
#     axs_item.grid()
#     axs_item.set_title(f'Feature {i}')
#
# plt.show()

交叉验证法
已经给出训练集和验证集，暂不用交叉验证

In [91]:
# from sklearn.model_selection import KFold, cross_val_score
#
# k_fold = KFold(n_splits=10)
#
# for train_indices, test_indices in k_fold.split(X):
#     print('Train: %s | test: %s' % (train_indices, test_indices))

## 平衡数据集

画出nan和null占比条形图
新数据集无需处理nan

In [92]:
# # 计算每列中NaN和null值的数量
# null_counts = df.isnull().sum()
# # 计算数据框中值的总数
# total_values = np.product(df.shape)
# # 计算每个列中NaN和null值的百分比
# null_percentages = null_counts / total_values * 100
#
# plt.figure(figsize=(20, 20))
# null_percentages.plot(kind='bar')

使用fillna填充平均值和中位数

In [93]:
df_fill = df.groupby('label').apply(lambda x: x.fillna(x.mean()))

提取出feature和label

In [94]:
feature_df = df_fill.drop(['sample_id', 'label'], axis=1)
labels_df = df_fill.label

合成少数类过采样技术（SMOTE）：在少数类中创建新的合成样本，以使其与多数类具有相同数量的样本。这种方法可以避免信息丢失和过拟合问题。您可以使用imblearn库中的SMOTE类来执行此操作。

In [95]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
transformed_feature_df, transformed_label_df = oversample.fit_resample(feature_df, labels_df)

查看每个label的feature数量

In [96]:
# print(f'new label count: {transformed_label_df.value_counts()}')
# print(f'old label count: {df.label.value_counts()}')

合并transformed_feature_df和transformed_label_df

In [97]:
transformed_df = pd.concat([transformed_label_df, transformed_feature_df], axis=1, join='outer')
transformed_df.to_csv("train_cleaned.csv")